## All imports:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.float_format', lambda x: '%.1f' % x)

## Working with data:

In [2]:
apple_store = pd.read_csv("appstore/AppleStore.csv")
apple_store_desc = pd.read_csv("appstore/appleStore_description.csv")
# importing all data for AppStore 


apple_store = apple_store.drop(['id', 'Unnamed: 0'], axis=1).astype({'size_bytes': 'int64',})\
.rename({'rating_count_tot': 'rating_count_total', 'user_rating': 'rating', 'track_name': 'app'}, axis=1)


In [3]:
#apple_store

In [ ]:
google_play_store = pd.read_csv("googleplay/googleplaystore.csv")
google_play_store_reviews = pd.read_csv("googleplay/googleplaystore_user_reviews.csv")
# importing all data for GooglePlay


for index in google_play_store.index:
    size = google_play_store.loc[index, 'Size']
    installs = google_play_store.loc[index, 'Installs']
    price = google_play_store.loc[index, 'Price']
    if installs.find("+") != -1:
        google_play_store.loc[index, 'Installs'] = installs[:-1].replace(",", "")
    if size.find("M") != -1:
        google_play_store.loc[index, 'Size'] = float(size[:-1]) * 125000 
        # removing "M" changing type from str to float and converting to bytes 
    elif size.find("k") != -1:
        google_play_store.loc[index, 'Size'] = float(size[:-1]) * 1000 
        # removing "k" changing type from str to float and converting to bytes 
    elif size.find('Varies') != -1:
        google_play_store.loc[index, 'Size'] = -1
    if price.find("$") != -1:
        google_play_store.loc[index, 'Price'] = price[1:]
        
        
google_play_store = google_play_store.astype({"Reviews": 'int32', "Rating": float, "Installs": 'int32', "Size": float, 'Price': float})\
.rename({'Size': 'size_bytes', 'Reviews': 'rating_count_total', 'Rating': 'rating', 'App': 'app', 'Current Ver': "ver", 'Price': 'price', 'Installs': 'installs'}, axis=1)


In [ ]:
google_play_store["rating"] = google_play_store["rating"].fillna(google_play_store["rating"].median())
#replacing all NaN values with median of column 'rating'

In [ ]:
#google_play_store[google_play_store['price'] > 60] # uncomment and run to see all expensive apps

In [ ]:
# all apps with price above 110 is just "i am rich" and "most expensive app" so i think we can easily remove them
google_play_store = google_play_store[google_play_store['price'] <= 80]

In [ ]:
#apple_store[apple_store['price'] > 60 ] # uncomment and run to see all expensive apps

In [ ]:
apple_store = apple_store[apple_store['price'] <= 80]

In [ ]:
# top rated apps for google where amount of reviews is more than 300
google_play_top = google_play_store[(google_play_store["rating_count_total"] > 300) & (google_play_store["rating"] >= 4.9)].sort_values(by=["rating_count_total"]) 


In [ ]:
# top rated apps for google where amount of reviews is more than 300
apple_top = apple_store[(apple_store["rating_count_total"] > 300) & (apple_store["rating"] >= 4.9)].sort_values(by=["rating_count_total"]) 


#### After cells above:
    google_play_store :
        * reviews, installs are now type:int32
        * rating, size, price are now type:float
        * size now is int and it is in bytes
        * rating - plus "+" sign removed
        * junk/scam apps like "i am rich" with price 400 removed
        
    apple_store : (нехватает конечно количества скачиваний но можно написать примерное количество исходя из соотношения кол-ва оценок/кол-во скачиваний в гуглплэй)
        *

#### Cell bellow is for playing with dataframes

In [ ]:
%matplotlib notebook

paid_apps_google = google_play_store[(google_play_store['price'] != 0.0)].fillna(0)
df1 = paid_apps_google.groupby("price").mean()

df2 = google_play_store[google_play_store['rating'].notna()].groupby("price").mean()
plt.title("Google play store grouped by price")
plt.ylabel('rating count')
plt.xlabel('price')

plt.ticklabel_format(style='plain')
plt.plot(df2.index, df2['installs'])

In [ ]:
%matplotlib notebook

paid_apps_apple = apple_store[(apple_store['price'] != 0.0) & apple_store['rating'] > 0.0] 
df3 = paid_apps_apple.groupby("price").mean()


df4 = apple_store.groupby("price").mean()
plt.title("Apple Store grouped by price")
plt.xlabel('price')
plt.ylabel('rating count')

plt.ticklabel_format(style='plain')
plt.plot(df4.index, df4['rating_count_total'])

In [ ]:
%matplotlib notebook

plt.ticklabel_format(style='plain')
sns.histplot(df1['rating'], kde=True, color='blue')
sns.histplot(df3['rating'], kde=True, color='orange')

In [ ]:
apple_store[apple_store['rating']>5]

In [ ]:
# some apps occure in table multiple times
google_play_store[google_play_store['app'] == 'Minecraft']
sns.pairplot(
    paid_apps_google,
    x_vars=['price', 'rating_count_total','size_bytes', 'installs'], # with 'price' it isn't working ¯\_(ツ)_/¯
    # and if change 'price' with for example  'rating_count_total' it'll give exception that price is not in index
    # also first chart isn't showing and i don't know why(if we just remove 'price')
    y_vars=["rating", 'price'],
)

In [ ]:
sns.pairplot(paid_apps_apple,
            x_vars=['price', 'rating_count_total','size_bytes', "lang.num"],
            y_vars=['rating', 'price'])